
# **IMPORTAÇÃO DOS PACOTES**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


### Criação do Dataframe com base no .CSV


In [2]:
dados = pd.read_csv('/content/household_power_consumption.txt', sep=';', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/household_power_consumption.txt'

1. Carregue o dataset e exiba as 10 primeiras linhas.


In [ ]:
dados.head(10)

2. Explique a diferença entre as variáveis Global_active_power e Global_reactive_power.


In [ ]:
# Explique a diferença entre as variáveis Global_active_power e Global_reactive_power
# Global_active_power: Esta variável representa a potência ativa total consumida pela residência em kilowatts (kW). É a potência real que realiza trabalho útil (como acender luzes, aquecer, etc.) e é a base para a cobrança na conta de luz.
# Global_reactive_power: Esta variável representa a potência reativa total em kilovolt-amperes reativos (kVAr). Esta potência está associada à criação e manutenção de campos eletromagnéticos em equipamentos (motores, transformadores) e não realiza trabalho útil diretamente.

3. Verifique se existem valores ausentes no dataset. Quantifique-os.


In [ ]:
dados.replace('?', np.nan, inplace=True)
dados = dados.apply(pd.to_numeric, errors='ignore')
print(dados.isnull().sum())


4. Converta a coluna Date para o tipo datetime e crie uma nova coluna com o dia da semana correspondente.

In [ ]:
dados['Date'] = pd.to_datetime(dados['Date'], format='%d/%m/%Y', errors='coerce')
dados['Day_of_Week'] = dados['Date'].dt.day_name()
print("As primeiras 5 linhas do dataset após a criação da coluna 'Day_of_Week':")
display(dados.head())

5. Filtre os registros apenas do ano de 2007 e calcule a média de consumo diário de Global_active_power.

In [ ]:
dados_2007 = dados[dados['Date'].dt.year == 2007]

dados_2007['Global_active_power'] = pd.to_numeric(dados_2007['Global_active_power'], errors='coerce')

dados_2007.dropna(subset=['Global_active_power'], inplace=True)

media_diaria_2007 = dados_2007.groupby(dados_2007['Date'].dt.date)['Global_active_power'].mean()

print("Média de consumo diário de Global_active_power em 2007:")
display(media_diaria_2007)

6. Gere um gráfico de linha mostrando a variação de Global_active_power em um único dia à sua escolha.

In [ ]:
dia_escolhido = '2007-01-01'
dados_dia_escolhido = dados[dados['Date'].dt.date == pd.to_datetime(dia_escolhido).date()].copy()

dados_dia_escolhido['Global_active_power'] = pd.to_numeric(dados_dia_escolhido['Global_active_power'], errors='coerce')

dados_dia_escolhido.dropna(subset=['Global_active_power'], inplace=True)


plt.figure(figsize=(12, 6))
plt.plot(dados_dia_escolhido['Time'], dados_dia_escolhido['Global_active_power'])
plt.title(f'Variação de Global_active_power em {dia_escolhido}')
plt.xlabel('Hora do Dia')
plt.ylabel('Global_active_power (kW)')
plt.xticks(dados_dia_escolhido['Time'][::60], rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

7. Crie um histograma da variável Voltage. O que pode ser observado sobre sua distribuição?


In [ ]:
dados['Voltage'] = pd.to_numeric(dados['Voltage'], errors='coerce')
plt.figure(figsize=(10,6))
plt.hist(dados['Voltage'].dropna(), bins=50, color='skyblue', edgecolor='black')
plt.title("Distribuição da variável Voltage")
plt.xlabel("Voltage (V)")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

8. Calcule o consumo médio por mês em todo o período disponível no dataset.


In [ ]:
dados['Date'] = pd.to_datetime(dados['Date'], format='%d/%m/%Y', errors='coerce')
dados['Year_Month'] = dados['Date'].dt.to_period('M')
dados.dropna(subset=['Global_active_power'], inplace=True)
media_mensal = dados.groupby('Year_Month')['Global_active_power'].mean()
print("Consumo médio de Global_active_power por mês:")
display(media_mensal)

9. Identifique o dia com maior consumo de energia ativa global (Global_active_power).


In [ ]:
consumo_diario_total = dados.groupby(dados['Date'].dt.date)['Global_active_power'].sum()
dia_maior_consumo = consumo_diario_total.idxmax()
maior_consumo_valor = consumo_diario_total.max()

print(f"O dia com maior consumo total de energia ativa global foi {dia_maior_consumo}, com {maior_consumo_valor:.2f} kW")

10. Compare o consumo médio de energia ativa global em dias de semana versus finais de
semana.

In [ ]:
dados['DayOfWeek'] = dados['Date'].dt.dayofweek
dados['Global_active_power'] = pd.to_numeric(dados['Global_active_power'], errors='coerce')

semana = dados[dados['DayOfWeek'] < 5]['Global_active_power'].mean()
fim_semana = dados[dados['DayOfWeek'] >= 5]['Global_active_power'].mean()

print(f"Consumo médio em dias de semana: {semana:.3f} kW")
print(f"Consumo médio em finais de semana: {fim_semana:.3f} kW")


11. Calcule a correlação entre as variáveis Global_active_power, Global_reactive_power, Voltage e Global_intensity.

In [ ]:
variaveis = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
dados_corr = dados[variaveis].apply(pd.to_numeric, errors='coerce')
correlacoes = dados_corr.corr()
print(correlacoes)


12. Crie uma nova variável chamada Total_Sub_metering que some Sub_metering_1, Sub_metering_2 e Sub_metering_3.

In [ ]:
dados[['Sub_metering_1','Sub_metering_2','Sub_metering_3']] = dados[['Sub_metering_1','Sub_metering_2','Sub_metering_3']].apply(pd.to_numeric, errors='coerce')
dados['Total_Sub_metering'] = dados['Sub_metering_1'] + dados['Sub_metering_2'] + dados['Sub_metering_3']
dados[['Sub_metering_1','Sub_metering_2','Sub_metering_3','Total_Sub_metering']].head()

13. Verifique se há algum mês em que Total_Sub_metering ultrapassa a média de Global_active_power.

In [ ]:
media_global = dados.groupby(dados['Year_Month'])['Global_active_power'].mean()
media_sub = dados.groupby(dados['Year_Month'])['Total_Sub_metering'].mean()

comparacao = media_sub > media_global
meses_superiores = comparacao[comparacao].index
print("Meses em que Total_Sub_metering superou a média de Global_active_power:")
print(meses_superiores)


14. Faça um gráfico de série temporal do Voltage para o ano de 2008.


In [ ]:
dados_2008 = dados[dados['Date'].dt.year == 2008].copy()
plt.figure(figsize=(12,6))
plt.plot(dados_2008['Date'], dados_2008['Voltage'], linewidth=0.5)
plt.title("Variação do Voltage em 2008")
plt.xlabel("Data")
plt.ylabel("Voltage (V)")
plt.grid(True)
plt.show()


15. Compare o consumo entre os meses de verão e inverno (no hemisfério norte).


In [ ]:
dados['Mes'] = dados['Date'].dt.month
dados['Ano'] = dados['Date'].dt.year

verao = dados[dados['Mes'].isin([6,7,8])]['Global_active_power'].mean()
inverno = dados[dados['Mes'].isin([12,1,2])]['Global_active_power'].mean()

print(f"Consumo médio no verão: {verao:.3f} kW")
print(f"Consumo médio no inverno: {inverno:.3f} kW")


16. Aplique uma amostragem aleatória de 1% dos dados e verifique se a distribuição de Global_active_power é semelhante à da base completa.

In [ ]:
dados_amostra = dados.sample(frac=0.01, random_state=42)

dados_amostra['Global_active_power'] = pd.to_numeric(dados_amostra['Global_active_power'], errors='coerce')

dados_amostra.dropna(subset=['Global_active_power'], inplace=True)

dados_amostra['Ano'] = dados_amostra['Date'].dt.year

dados_filtrados_amostra = dados_amostra[dados_amostra['Ano'].isin([2007, 2008])]


plt.figure(figsize=(10,6))
dados_filtrados_amostra.boxplot(column='Global_active_power', by='Ano')
plt.title("Comparação Global_active_power entre 2007 e 2008 (Amostra de 1%)")
plt.suptitle("")
plt.xlabel("Ano")
plt.ylabel("Global_active_power (kW)")
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(dados['Global_active_power'].dropna(), bins=50, color='blue', label='Base Completa', kde=True)
sns.histplot(dados_amostra['Global_active_power'].dropna(), bins=50, color='red', label='Amostra (1%)', kde=True)
plt.title("Comparação da Distribuição de Global_active_power (Base Completa vs Amostra)")
plt.xlabel("Global_active_power (kW)")
plt.ylabel("Frequência")
plt.legend()
plt.show()

17. Utilize uma técnica de normalização (Min-Max Scaling) para padronizar as variáveis numéricas principais.

In [ ]:
media_anual = dados.groupby(dados['Date'].dt.year)['Global_active_power'].mean()
media_anual.plot(kind='line', marker='o', figsize=(10,6))
plt.title("Tendência de Global_active_power ao longo dos anos")
plt.xlabel("Ano")
plt.ylabel("Consumo médio (kW)")
plt.grid(True)
plt.show()


### 18. Aplique K-means para segmentar os dias em 3 grupos distintos de consumo elétrico. Interprete os resultados.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

consumo_diario = dados.groupby('Date')['Global_active_power'].sum().dropna().reset_index()

scaler = StandardScaler()
consumo_diario['Global_active_power_scaled'] = scaler.fit_transform(consumo_diario[['Global_active_power']])

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
consumo_diario['Cluster'] = kmeans.fit_predict(consumo_diario[['Global_active_power_scaled']])

date_to_cluster = dict(zip(consumo_diario['Date'], consumo_diario['Cluster']))

dados['Cluster'] = dados['Date'].map(date_to_cluster)

media_consumo_por_cluster = dados.groupby('Cluster')['Global_active_power'].mean()

print("Consumo médio por cluster:")
print(media_consumo_por_cluster)

plt.figure(figsize=(10, 6))
sns.histplot(data=dados.dropna(subset=['Cluster']), x='Global_active_power', hue='Cluster', bins=50, kde=True)
plt.title("Distribuição de Global_active_power por Cluster")
plt.xlabel("Global_active_power (kW)")
plt.ylabel("Frequência")
plt.show()

### 19. Realize uma decomposição de série temporal (tendência, sazonalidade e resíduo) para Global_active_power em um período de 6 meses.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

dados_diarios = dados.groupby('Date')['Global_active_power'].mean().dropna()

serie_temporal_6meses = dados_diarios['2007-01-01':'2007-06-30']

decomposicao = seasonal_decompose(serie_temporal_6meses, model='additive', period=7)

fig = decomposicao.plot()
fig.set_size_inches(12, 8)
plt.suptitle("Decomposição da Série Temporal de Global_active_power (Jan-Jun 2007)")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### 20. Treine um modelo de regressão linear simples para prever Global_active_power a partir de Global_intensity. Avalie o erro do modelo.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

dados_regressao = dados[['Global_active_power', 'Global_intensity']].dropna()

X = dados_regressao[['Global_intensity']]
y = dados_regressao['Global_active_power']

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

modelo_regressao = LinearRegression()
modelo_regressao.fit(X_treino, y_treino)

y_previsoes = modelo_regressao.predict(X_teste)

mse = mean_squared_error(y_teste, y_previsoes)
rmse = np.sqrt(mse)
r2 = r2_score(y_teste, y_previsoes)

print(f"Erro Quadrático Médio (MSE): {mse:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")

plt.figure(figsize=(10, 6))
plt.scatter(X_teste, y_teste, color='blue', label='Valores Reais', alpha=0.5)
plt.plot(X_teste, y_previsoes, color='red', linewidth=2, label='Previsões do Modelo')
plt.title("Regressão Linear: Global_active_power vs Global_intensity")
plt.xlabel("Global_intensity (A)")
plt.ylabel("Global_active_power (kW)")
plt.legend()
plt.show()

## PARTE 2 – Exercícios adicionais no dataset inicial

### 21. Séries temporais por hora
- Converta Date e Time em índice datetime.
- Reamostre os dados em intervalos de 1 hora, calculando a média de Global_active_power.
- Identifique os horários de maior consumo médio ao longo do dia.

In [ ]:
dados['Datetime'] = pd.to_datetime(dados['Date'].astype(str) + ' ' + dados['Time'])
dados.set_index('Datetime', inplace=True)

dados_horarios = dados['Global_active_power'].resample('h').mean().dropna()

consumo_medio_por_hora = dados_horarios.groupby(dados_horarios.index.hour).mean()

print("Consumo médio de Global_active_power por hora do dia:")
display(consumo_medio_por_hora)

plt.figure(figsize=(10, 6))
consumo_medio_por_hora.plot(kind='bar')
plt.title('Consumo Médio de Global_active_power por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Consumo Médio (kW)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.show()

### 22. Autocorrelação do consumo
- Use a série temporal de Global_active_power.
- Calcule a autocorrelação em lags de 1h, 24h e 48h.
- Pergunta: existem padrões repetidos diariamente?

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf


lags = [1, 24, 48]
autocorrelacoes = {}

for lag in lags:
    series_lagged = dados_horarios.copy().dropna()
    autocorrelation = series_lagged.autocorr(lag=lag)
    autocorrelacoes[f'Lag {lag}h'] = autocorrelation

print("Autocorrelação de Global_active_power:")
for lag, corr in autocorrelacoes.items():
    print(f"{lag}: {corr:.2f}")

plt.figure(figsize=(12, 6))
plot_acf(dados_horarios.dropna(), lags=72)
plt.title('Função de Autocorrelação (ACF) de Global_active_power (Dados Horários)')
plt.xlabel('Lag (Horas)')
plt.ylabel('Autocorrelação')
plt.grid(True)
plt.show()

# Pergunta: existem padrões repetidos diariamente?
# Resposta baseada na análise da ACF plotada acima:
# Se a ACF mostra picos significativos em lags que são múltiplos de 24 (24h, 48h, etc.),
# isso indica que há uma forte correlação entre o consumo de energia em um determinado
# momento e o consumo 24 ou 48 horas antes, sugerindo um padrão diário repetido.

### 23. Redução de dimensionalidade com PCA
- Selecione Global_active_power, Global_reactive_power, Voltage e Global_intensity.
- Aplique PCA para reduzir para 2 componentes principais.
- Analise a variância explicada por cada componente.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

variaveis_pca = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
dados_pca = dados[variaveis_pca].dropna()

scaler_pca = StandardScaler()
dados_pca_scaled = scaler_pca.fit_transform(dados_pca)

pca = PCA(n_components=2)
componentes_principais = pca.fit_transform(dados_pca_scaled)

componentes_df = pd.DataFrame(data = componentes_principais, columns = ['componente_principal_1', 'componente_principal_2'])

variancia_explicada = pca.explained_variance_ratio_
print(f"Variância explicada pelo Componente Principal 1: {variancia_explicada[0]:.2f}")
print(f"Variância explicada pelo Componente Principal 2: {variancia_explicada[1]:.2f}")
print(f"Variância total explicada pelos 2 Componentes Principais: {variancia_explicada.sum():.2f}")

print("\nPrimeiras 5 linhas dos componentes principais:")
display(componentes_df.head())

### 24. Visualização de clusters no espaço PCA
- Combine os resultados do PCA com K-Means (3 clusters).
- Plote os pontos resultantes e pinte cada grupo por cluster.
- Pergunta: os grupos se separam de forma clara?

In [ ]:
indices_pca = dados_pca.index

if 'Cluster' in dados.columns:
    clusters_para_pca = dados.loc[indices_pca, 'Cluster']

    componentes_df_reset = componentes_df.reset_index(drop=True)
    clusters_para_pca_reset = clusters_para_pca.reset_index(drop=True)

    componentes_df_reset['Cluster'] = clusters_para_pca_reset

    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='componente_principal_1', y='componente_principal_2', hue='Cluster', data=componentes_df_reset.dropna(subset=['Cluster']), palette='viridis', alpha=0.6)
    plt.title('Visualização dos Clusters no Espaço PCA')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.grid(True)
    plt.show()

    # Pergunta: os grupos se separam de forma clara?
    # Resposta baseada na visualização:
    # Observar se os pontos de diferentes cores (clusters) formam agrupamentos distintos
    # no gráfico de dispersão. Se houver sobreposição significativa, os clusters podem
    # não estar bem separados neste espaço bidimensional.
else:
    print("A coluna 'Cluster' não foi encontrada no DataFrame 'dados'. Por favor, execute a Questão 18 primeiro.")

### 25. Regressão polinomial vs linear
- Modele Global_active_power em função de Voltage.
- Compare Regressão Linear Simples com Regressão Polinomial (grau 2).
- Analise RMSE e a curva ajustada.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

dados_regressao_voltage = dados[['Global_active_power', 'Voltage']].dropna()

X_voltage = dados_regressao_voltage[['Voltage']]
y_voltage = dados_regressao_voltage['Global_active_power']

X_treino_v, X_teste_v, y_treino_v, y_teste_v = train_test_split(X_voltage, y_voltage, test_size=0.2, random_state=42)

modelo_linear = LinearRegression()
modelo_linear.fit(X_treino_v, y_treino_v)
y_previsoes_linear = modelo_linear.predict(X_teste_v)
rmse_linear = np.sqrt(mean_squared_error(y_teste_v, y_previsoes_linear))

print(f"Regressão Linear Simples - RMSE: {rmse_linear:.2f}")

poly_features = PolynomialFeatures(degree=2)
X_treino_poly = poly_features.fit_transform(X_treino_v)
X_teste_poly = poly_features.transform(X_teste_v)

modelo_polinomial = LinearRegression()
modelo_polinomial.fit(X_treino_poly, y_treino_v)
y_previsoes_polinomial = modelo_polinomial.predict(X_teste_poly)
rmse_polinomial = np.sqrt(mean_squared_error(y_teste_v, y_previsoes_polinomial))

print(f"Regressão Polinomial (grau 2) - RMSE: {rmse_polinomial:.2f}")

plt.figure(figsize=(10, 6))
plt.scatter(X_teste_v, y_teste_v, color='blue', label='Valores Reais', alpha=0.5)

X_plot = pd.DataFrame(np.linspace(X_teste_v['Voltage'].min(), X_teste_v['Voltage'].max(), 100), columns=['Voltage'])


y_plot_linear = modelo_linear.predict(X_plot)
plt.plot(X_plot['Voltage'], y_plot_linear, color='red', linewidth=2, label='Regressão Linear')

X_plot_poly = poly_features.transform(X_plot)
y_plot_polinomial = modelo_polinomial.predict(X_plot_poly)
plt.plot(X_plot['Voltage'], y_plot_polinomial, color='green', linewidth=2, label='Regressão Polinomial (grau 2)')

plt.title("Comparação entre Regressão Linear e Polinomial (Voltage vs Global_active_power)")
plt.xlabel("Voltage (V)")
plt.ylabel("Global_active_power (kW)")
plt.legend()
plt.grid(True)
plt.show()

### 26. Carregamento e inspeção inicial
- Carregue o dataset no Pandas.
- Liste tipos de dados e estatísticas descritivas (.info() e .describe()).

In [ ]:
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv'

dados_energia = pd.read_csv(dataset_url)

print("Informações do Dataset:")
dados_energia.info()

print("\nEstatísticas Descritivas do Dataset:")
display(dados_energia.describe())

### Preparação de dados
- Converter a coluna 'date' para o tipo datetime.

In [ ]:
dados_energia['date'] = pd.to_datetime(dados_energia['date'])

print("Tipos de dados após a conversão da coluna 'date':")
print(dados_energia.info())

### 27. Distribuição do consumo
- Crie histogramas e séries temporais para a variável Appliances.
- Pergunta: o consumo tende a se concentrar em valores baixos ou altos?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(dados_energia['Appliances'], bins=50, kde=True)
plt.title('Distribuição do Consumo de Energia (Appliances)')
plt.xlabel('Consumo de Energia (Wh)')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

dados_energia.set_index('date', inplace=True)
appliances_horario = dados_energia['Appliances'].resample('h').mean()

plt.figure(figsize=(15, 7))
appliances_horario.plot()
plt.title('Série Temporal do Consumo Médio de Energia (Appliances) por Hora')
plt.xlabel('Data')
plt.ylabel('Consumo Médio de Energia (Wh)')
plt.grid(True)
plt.show()

# Pergunta: o consumo tende a se concentrar em valores baixos ou altos?
# Resposta baseada nos gráficos:
# Observando o histograma, verifique onde a maioria dos valores de consumo se agrupa (mais à esquerda para valores baixos, mais à direita para valores altos).

### 28. Correlações com variáveis ambientais
- Calcule correlações entre Appliances e variáveis como temperatura e umidade.
- Pergunta: quais fatores têm mais relação com o consumo?

In [ ]:
variaveis_ambientais = ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5',
                        'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
                        'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']

correlacoes_ambientais = dados_energia[['Appliances'] + variaveis_ambientais].corr()['Appliances'].sort_values(ascending=False)

print("Correlações de Appliances com variáveis ambientais:")
display(correlacoes_ambientais)

# Pergunta: quais fatores têm mais relação com o consumo?
# Resposta baseada nos resultados da correlação:
# Os fatores com os maiores valores absolutos de correlação (mais próximos de 1 ou -1)
# são os que têm mais relação com o consumo. Analisar os valores e identificar quais
# variáveis ambientais apresentam as correlações mais fortes.

### 29. Normalização dos dados
- Aplique Min-Max Scaling às variáveis numéricas.
- Reutilize esses dados em modelos posteriores.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

colunas_numericas = dados_energia.select_dtypes(include=np.number).columns.tolist()
colunas_para_normalizar = [col for col in colunas_numericas if col not in ['Appliances', 'lights']]

dados_energia_scaled = dados_energia.copy()

scaler = MinMaxScaler()

dados_energia_scaled[colunas_para_normalizar] = scaler.fit_transform(dados_energia_scaled[colunas_para_normalizar])

print("Primeiras 5 linhas do DataFrame com variáveis numéricas normalizadas:")
display(dados_energia_scaled.head())

print("\nEstatísticas descritivas das variáveis numéricas normalizadas:")
display(dados_energia_scaled[colunas_para_normalizar].describe())

### 30. PCA
- Aplique PCA e reduza para 2 componentes principais.
- Plote os dados resultantes.
- Pergunta: aparecem padrões ou agrupamentos naturais?

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

variaveis_pca_energia = dados_energia_scaled.select_dtypes(include=np.number).columns.tolist()
variaveis_para_pca = [col for col in variaveis_pca_energia if col not in ['Appliances', 'lights']]

dados_para_pca = dados_energia_scaled[variaveis_para_pca].copy()

pca_energia = PCA(n_components=2)
componentes_principais_energia = pca_energia.fit_transform(dados_para_pca)

componentes_df_energia = pd.DataFrame(data = componentes_principais_energia,
                                      columns = ['componente_principal_1', 'componente_principal_2'])

componentes_df_energia['Appliances'] = dados_energia_scaled['Appliances'].values

plt.figure(figsize=(10, 8))
sns.scatterplot(x='componente_principal_1', y='componente_principal_2', hue='Appliances', data=componentes_df_energia, palette='viridis', alpha=0.6, s=10) # s controla o tamanho dos pontos
plt.title('Visualização dos Dados no Espaço PCA (Colorido por Consumo de Energia)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid(True)
plt.show()

variancia_explicada_energia = pca_energia.explained_variance_ratio_
print(f"Variância explicada pelo Componente Principal 1: {variancia_explicada_energia[0]:.2f}")
print(f"Variância explicada pelo Componente Principal 2: {variancia_explicada_energia[1]:.2f}")
print(f"Variância total explicada pelos 2 Componentes Principais: {variancia_explicada_energia.sum():.2f}")


# Pergunta: aparecem padrões ou agrupamentos naturais?
# Resposta baseada na visualização:
# Observar o scatter plot. Se os pontos formarem aglomerados distintos, isso pode indicar agrupamentos naturais.
# A coloração por 'Appliances' pode mostrar se há uma separação visual clara entre diferentes níveis de consumo no espaço PCA.

### 31. Regressão Linear Múltipla
- Modele Appliances em função das variáveis ambientais.
- Avalie R² e erro médio.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

variaveis_ambientais_scaled = [col for col in dados_energia_scaled.select_dtypes(include=np.number).columns if col not in ['Appliances', 'lights']]
X = dados_energia_scaled[variaveis_ambientais_scaled]
y = dados_energia_scaled['Appliances']

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

modelo_regressao_linear = LinearRegression()
modelo_regressao_linear.fit(X_treino, y_treino)

y_previsoes_linear = modelo_regressao_linear.predict(X_teste)

mse_linear = mean_squared_error(y_teste, y_previsoes_linear)
rmse_linear = np.sqrt(mse_linear)
r2_linear = r2_score(y_teste, y_previsoes_linear)

print(f"Regressão Linear Múltipla:")
print(f"Erro Quadrático Médio (MSE): {mse_linear:.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_linear:.2f}")
print(f"Coeficiente de Determinação (R²): {r2_linear:.2f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_teste.head(100), y_previsoes_linear[:100], color='blue', alpha=0.6)
plt.plot([y_teste.min(), y_teste.max()], [y_teste.min(), y_teste.max()], 'k--', lw=2)
plt.title("Regressão Linear Múltipla: Valores Reais vs Previsões (Amostra)")
plt.xlabel("Valores Reais de Appliances")
plt.ylabel("Previsões de Appliances")
plt.grid(True)
plt.show()

### 32. Random Forest Regressor
- Treine um modelo de Random Forest para prever Appliances.
- Compare o RMSE com a regressão linear.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_rf.fit(X_treino, y_treino)

y_previsoes_rf = modelo_rf.predict(X_teste)

mse_rf = mean_squared_error(y_teste, y_previsoes_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Random Forest Regressor - RMSE: {rmse_rf:.2f}")

print(f"Regressão Linear Múltipla - RMSE: {rmse_linear:.2f}")

# Pergunta: Como o RMSE se compara?
# Resposta: Comparar os valores de RMSE. Um RMSE menor indica um modelo com melhor desempenho (menor erro de previsão).

### 33. K-Means clustering
- Aplique K-Means com 3 a 5 clusters.
- Interprete os perfis de consumo.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

variaveis_para_kmeans = [col for col in dados_energia_scaled.select_dtypes(include=np.number).columns if col not in ['Appliances', 'lights', 'rv1', 'rv2']]

dados_para_kmeans = dados_energia_scaled[variaveis_para_kmeans].copy()

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
dados_energia_scaled['Cluster_KMeans'] = kmeans.fit_predict(dados_para_kmeans)

dados_energia['Cluster_KMeans'] = dados_energia_scaled['Cluster_KMeans']

perfil_clusters = dados_energia.groupby('Cluster_KMeans')[['Appliances'] + variaveis_para_kmeans].mean()

print(f"Perfis de consumo para {n_clusters} clusters:")
display(perfil_clusters)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster_KMeans', y='Appliances', data=dados_energia)
plt.title(f'Distribuição de Appliances por Cluster ({n_clusters} Clusters)')
plt.xlabel('Cluster')
plt.ylabel('Consumo de Energia (Wh)')
plt.show()



### 34. Classificação binária
- Crie uma variável: alto vs baixo consumo (Appliances maior/menor que a mediana).
- Treine Logistic Regression e Random Forest Classifier.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

mediana_appliances = dados_energia_scaled['Appliances'].median()
dados_energia_scaled['Consumo_Alto'] = (dados_energia_scaled['Appliances'] > mediana_appliances).astype(int)

variaveis_para_classificacao = [col for col in dados_energia_scaled.select_dtypes(include=np.number).columns if col not in ['Appliances', 'Consumo_Alto']]
X_class = dados_energia_scaled[variaveis_para_classificacao]
y_class = dados_energia_scaled['Consumo_Alto']

X_treino_class, X_teste_class, y_treino_class, y_teste_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42, stratify=y_class) # Usar stratify para manter a proporção das classes

modelo_regressao_logistica = LogisticRegression(random_state=42, max_iter=1000)
modelo_regressao_logistica.fit(X_treino_class, y_treino_class)

modelo_rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf_classifier.fit(X_treino_class, y_treino_class)

print("Modelos de classificação treinados: Regressão Logística e Random Forest Classifier.")

### 35. Avaliação de classificação
- Gere matriz de confusão e métricas (accuracy, precision, recall, F1-score).
- Pergunta: o modelo erra mais para alto ou para baixo consumo?

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_previsoes_logistica = modelo_regressao_logistica.predict(X_teste_class)

print("Avaliação do Modelo de Regressão Logística:")
print(f"Acurácia: {accuracy_score(y_teste_class, y_previsoes_logistica):.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_teste_class, y_previsoes_logistica))
print("\nMatriz de Confusão:")
cm_logistica = confusion_matrix(y_teste_class, y_previsoes_logistica)
sns.heatmap(cm_logistica, annot=True, fmt='d', cmap='Blues', xticklabels=['Baixo Consumo', 'Alto Consumo'], yticklabels=['Baixo Consumo', 'Alto Consumo'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()

y_previsoes_rf_class = modelo_rf_classifier.predict(X_teste_class)

print("\n" + "="*50 + "\n")
print("Avaliação do Modelo Random Forest Classifier:")
print(f"Acurácia: {accuracy_score(y_teste_class, y_previsoes_rf_class):.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_teste_class, y_previsoes_rf_class))
print("\nMatriz de Confusão:")
cm_rf = confusion_matrix(y_teste_class, y_previsoes_rf_class)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', xticklabels=['Baixo Consumo', 'Alto Consumo'], yticklabels=['Baixo Consumo', 'Alto Consumo'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Random Forest Classifier')
plt.show()

# Pergunta: o modelo erra mais para alto ou para baixo consumo?
# Resposta baseada nas matrizes de confusão:
# Analisar as matrizes de confusão.
# Erros para "Alto Consumo" (Falsos Negativos): prevê Baixo, mas é Alto (canto inferior esquerdo da matriz).
# Erros para "Baixo Consumo" (Falsos Positivos): prevê Alto, mas é Baixo (canto superior direito da matriz).
# Comparar o número de Falsos Negativos e Falsos Positivos para cada modelo.

36. Importação e visualização inicial

In [ ]:
# São 9 variáveis e 2075259 registros.

37. Amostragem de dados (1%)




In [ ]:
# Sim, são semelhantes.

38. Distribuição do consumo


In [ ]:
# Concentrada em valores baixos.

39. Relação entre variáveis elétricas


In [ ]:
# Existe, quanto maior a Global_active_power, menor e mais estável é a voltagem.

40. Clustering com K-Means


In [ ]:
# Sim, a distribuição dos sub_metering são distintas em cada cenário, oscilando bastante, mesmo o 1 e 2 sendo mais parecidos.